In [1]:
##TO RUN THE FILE GOOGLE COLAB
import os
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# os.environ["SPARK_HOME"] = "/content/spark-2.4.0-bin-hadoop2.7"

import findspark
findspark.init('/home/cse587/spark-2.4.0-bin-hadoop2.7')     ##TO RUN THE FILE ON
# findspark.init()                                           ##TO RUN THE FILE GOOGLE COLAB
from pyspark.sql import SparkSession
# spark = SparkSession.builder.master("local[*]").getOrCreate()
spark = SparkSession.builder.master("local[*]")\
        .config("spark.executor.memory", "2g")\
        .config("spark.driver.memory", "2g")\
        .config("spark.memory.offHeap.enabled",'true')\
        .config("spark.memory.offHeap.size","2g")\
        .getOrCreate()

!ls

CSE_487_587_Assignment_3___Predictive_Analytics_with_Apache_Spark.pdf
Data
Desktop
DIC
DIC_PySpark_Part3.ipynb
Documents
Downloads
examplehadoop
examples.desktop
genre_lables.csv
gutenberg
hadoop-2.8.5.tar.gz
hadoop-3.1.2
hadoop-3.1.2.tar.gz
hadooptmpdata
hdfs
hs_err_pid10739.log
hs_err_pid24575.log
hs_err_pid24640.log
mapping.csv
Music
part3
Pictures
Public
sample.csv
spark-2.4.0-bin-hadoop2.7
spark-2.4.0-bin-hadoop2.7.tgz
Templates
test.csv
train.csv
Untitled-1.ipynb
Untitled.ipynb
Videos


In [2]:
import numpy as np
import pandas as pd
from ast import literal_eval 
import pyspark.sql.functions as F
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType, IntegerType, ArrayType
from pyspark.ml import Pipeline
from pyspark.ml.feature import StopWordsRemover
from pyspark.ml.feature import Tokenizer, RegexTokenizer
from pyspark.ml.feature import HashingTF, IDF
from pyspark.sql.functions import regexp_replace,col,array_contains,explode
from pyspark.sql.functions import split
from pyspark.sql.functions import monotonically_increasing_id
from functools import reduce
from pyspark.sql.functions import concat_ws
from pyspark.ml.classification import LogisticRegression,LogisticRegressionModel
from pyspark.ml.feature import Word2Vec

In [3]:
pd_df = pd.read_csv(r'train.csv')
data_spark_df = spark.createDataFrame(pd_df)
pd_df['genre']= pd_df['genre'].apply(literal_eval)
all_genre = pd_df['genre'].to_list()
names =['Drama','Comedy','Romance Film','Thriller','Action','World cinema','Crime Fiction','Horror','Black-and-white','Indie','Action/Adventure','Adventure','Family Film','Short Film','Romantic drama','Animation','Musical','Science Fiction','Mystery','Romantic comedy']
matrix = np.zeros((len(pd_df),len(names)))

for i,genre in enumerate(all_genre):
  for j,g in enumerate(genre):
    for k,name in enumerate(names):
        if name==g:
          matrix[i][k] = 1
names = "Drama , Comedy , Romance Film , Thriller , Action , World cinema , Crime Fiction , Horror , Black-and-white , Indie , Action/Adventure , Adventure , Family Film , Short Film , Romantic drama , Animation , Musical , Science Fiction , Mystery , Romantic comedy"
np.savetxt("genre_lables.csv", matrix, delimiter=",",fmt='%d',header=names)


In [4]:
lables_df = pd.read_csv(r'genre_lables.csv')
test_pd_df = pd.read_csv(r'test.csv')
lables_spark_df = spark.createDataFrame(lables_df)
test_spark_df = spark.createDataFrame(test_pd_df)
ddf1 = data_spark_df.withColumn("row_id", monotonically_increasing_id())
ddf2 = lables_spark_df.withColumn("row_id", monotonically_increasing_id())
ddf3 = test_spark_df.withColumn("row_id", monotonically_increasing_id())
df = ddf1.join(ddf2, "row_id").drop("row_id")
test_df = ddf3.join(ddf2, "row_id").drop("row_id")

In [5]:
%%time
tokenizer = Tokenizer(inputCol="plot", outputCol="words")
word2Vec = Word2Vec(inputCol="words", outputCol="features", minCount=1)
pipeline = Pipeline(stages=[tokenizer, word2Vec])

model = pipeline.fit(df)
dataset = model.transform(df)

model2 = pipeline.fit(test_df)
test_dataset = model2.transform(test_df)

CPU times: user 109 ms, sys: 23.9 ms, total: 132 ms
Wall time: 5min 43s


In [6]:
dfList = []
labelCols = lables_spark_df.columns

In [ ]:
%%time
##Note: Run this cell if you wann train the model again
lr = LogisticRegression(featuresCol = 'features',maxIter=750)
for labelCol in labelCols:
    lr.setLabelCol(labelCol)
    lrModel = lr.fit(dataset)
    lrModel.save(r"part3/"+labelCol) ##""" if you want train the model again comment this line and uncomment the next following line"""
    # lrModel.write().overwrite().save(r"part3/"+labelCol)

In [7]:
%%time
from pyspark.ml.classification import LogisticRegression,LogisticRegressionModel
for labelCol in labelCols:
    lrModel2 = LogisticRegressionModel.load("part3/"+labelCol)
    predictions = lrModel2.transform(test_dataset)
    predictions = predictions.drop("features",labelCol,"rawPrediction","probability")
    predictions = predictions.withColumn("prediction",F.col("prediction").cast(IntegerType()))
    dfList.append(predictions.select("movie_id","prediction"))

CPU times: user 443 ms, sys: 59.7 ms, total: 503 ms
Wall time: 12.1 s


In [8]:
%%time
dfs_renamed = [df.selectExpr('movie_id', f'prediction as prediction_{i}') for i, df in enumerate(dfList)]
temp_df = reduce(lambda x, y: x.join(y, ['movie_id'], how='full'), dfs_renamed)
col_list = ['prediction_%d' % i for i in range(len(dfList))]
temp_df = temp_df.withColumn('predictions',concat_ws(" ",*col_list)).drop(*col_list).toPandas().to_csv("predictions_part3.csv",index=False)

CPU times: user 709 ms, sys: 142 ms, total: 851 ms
Wall time: 3min 20s
